In [ ]:
import requests
from typing import Dict, Any, Optional

BASE_URL = "https://berghain.challenges.listenlabs.ai/"  # ← replace with the real host
PLAYER_ID = "5d7389b8-683e-453b-a35f-6d21b1b5e94e"
SCENARIO = 3  # choose 1, 2, or 3

def create_new_game(scenario: int, player_id: str) -> Dict[str, Any]:
    url = f"{BASE_URL}/new-game"
    res = requests.get(url, params={"scenario": scenario, "playerId": player_id}, timeout=30)
    res.raise_for_status()
    data = res.json()
    print("New game:", data["gameId"])
    return data



In [2]:
def decide_and_next(game_id: str, person_index: int, accept: Optional[bool] = None) -> Dict[str, Any]:
    url = f"{BASE_URL}/decide-and-next"
    params = {"gameId": game_id, "personIndex": person_index}
    if accept is not None:
        params["accept"] = str(accept).lower()  # true/false in query string
    res = requests.get(url, params=params, timeout=30)
    res.raise_for_status()
    return res.json()

new_game = create_new_game(SCENARIO, PLAYER_ID)
game_id = new_game["gameId"]
constraints = new_game.get("constraints", [])
stats = new_game.get("attributeStatistics", {})
print(constraints)
print(stats)

New game: 010aff47-127c-4fc3-8092-8f28941de814
[{'attribute': 'underground_veteran', 'minCount': 500}, {'attribute': 'international', 'minCount': 650}, {'attribute': 'fashion_forward', 'minCount': 550}, {'attribute': 'queer_friendly', 'minCount': 250}, {'attribute': 'vinyl_collector', 'minCount': 200}, {'attribute': 'german_speaker', 'minCount': 800}]
{'relativeFrequencies': {'underground_veteran': 0.6794999999999999, 'international': 0.5735, 'fashion_forward': 0.6910000000000002, 'queer_friendly': 0.04614, 'vinyl_collector': 0.044539999999999996, 'german_speaker': 0.4565000000000001}, 'correlations': {'underground_veteran': {'underground_veteran': 1, 'international': -0.08110175777152992, 'fashion_forward': -0.1696563475505309, 'queer_friendly': 0.03719928376753885, 'vinyl_collector': 0.07223521156389842, 'german_speaker': 0.11188766703422799}, 'international': {'underground_veteran': -0.08110175777152992, 'international': 1, 'fashion_forward': 0.375711059360155, 'queer_friendly': 0.0

In [3]:
state = decide_and_next(game_id, person_index=0)
if state.get("status") == "failed":
    raise RuntimeError(f"Game failed: {state.get('reason')}")
admitted = state.get("admittedCount", 0)
rejected = state.get("rejectedCount", 0)
next_person = state.get("nextPerson")
total_gsint = 0
total_qfvinco = 0
total_qfgs = 0
total_gs = 0
total_int = 0
n_accepted = 0
while state.get("status") == "running" and next_person:
        idx = next_person["personIndex"]
        attrs = next_person["attributes"]  # { attributeId: bool }
        ugv, int, ff, qf, vinco, gs = (
            attrs.get("underground_veteran", False), 
            attrs.get("international", False), 
            attrs.get("fashion_forward", False), 
            attrs.get("queer_friendly", False), 
            attrs.get("vinyl_collector", False), 
            attrs.get("german_speaker", False)
        )
        accept = False
        if gs and int and (ff or ugv):
            accept = True
            total_gsint += 1
        elif (total_qfvinco < 200) or (total_qfgs < 250):
            if qf and vinco and int and total_qfvinco < 200:
                accept = True
                total_qfvinco += 1
            if qf and vinco and gs and total_qfgs < 250:
                accept = True
                total_qfgs += 1
        else:
            if gs and (ff or ugv) and total_int >= 650:
                accept = True
            elif int and (ff or ugv) and total_gs >= 800:
                accept = True            
        if accept:   
            total_gs += gs
            total_int += int
        state = decide_and_next(game_id, person_index=idx, accept=accept)
        status = state.get("status")
        if status == "failed":
            raise RuntimeError(f"Game failed: {state.get('reason')}")

        admitted = state.get("admittedCount", admitted)
        rejected = state.get("rejectedCount", rejected)
        next_person = state.get("nextPerson")

        print(f"Person {idx}: accept={accept} | admitted={admitted} rejected={rejected}")

        # 5) done
if state.get("status") == "completed":
        print("Game completed.")
        print("Final rejected count:", state.get("rejectedCount"))
else:
        print("Game ended with state:", state)

Person 0: accept=False | admitted=0 rejected=1
Person 1: accept=False | admitted=0 rejected=2
Person 2: accept=False | admitted=0 rejected=3


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))